## 동대문구 불법주정차 데이터를 이용한 데이터 분석
### 1. 동대문구 내 동네별 현황

In [25]:
import pandas as pd


# Load the data from the uploaded CSV file
dongdaemoon_file_path = 'C:/Users/Administrator/Desktop/dayeong/불법주정차_신고현황_동대문구_filtered.csv'
dongdaemoon_df = pd.read_csv(dongdaemoon_file_path)

dongdaemoon_df.head(10)

,민원접수일,민원접수시간,주소,경도,위도,도로명주소
0,20240208,07:55:00,서울특별시 동대문구 외대역동로 90,127.065608,37.596081,이문동 이문2동 동대문구 서울특별시
1,20240223,19:23:00,서울특별시 동대문구 전농로 50-1,127.057294,37.569414,바지락칼국수 전농로 답십리2동 동대문구 서울특별시
2,20240218,19:32:00,서울 동대문구 회기로26길 2863,127.053621,37.590506,회기로18길 회기동 동대문구 서울특별시
3,20231221,10:59:00,동대문구 한천로 185 동트리움과 파리바게트 사이,127.067800,37.577544,전곡시장앞 한천로 장안2동 동대문구 서울특별시
4,20231106,15:36:00,서울특별시 동대문구 홍릉로 1-2,127.044925,37.580479,홍릉로 청량리동 동대문구 서울특별시
5,20240129,17:46:00,서울특별시 동대문구 휘경로 13,127.061888,37.596220,26 이문로30길 외대 모두의거리 이문2동 동대문구 서울특별시
6,20231210,14:33:00,장한로173,127.071729,37.575916,장한로 장안2동 동대문구 서울특별시
7,20231211,14:42:00,서울특별시 동대문구 천호대로22길 33,127.033311,37.572392,천호대로22길 용두동 용신동 동대문구 서울특별시
8,20231123,22:06:00,서울특별시 동대문구 사가정로 207,127.068344,37.579014,한천로 장안2동 동대문구 서울특별시
9,20240105,10:56:00,서울특별시 동대문구 무학로 180,127.030199,37.580279,무학로 용두동 용신동 동대문구 서울특별시


## 지도 시각화 (folium)

In [26]:
import folium
import requests
import json

# reference
# https://m.blog.naver.com/PostView.naver?blogId=peacenuts&logNo=222408862138&categoryNo=7&proxyReferer=

In [46]:
# 지도생성 (동대문구 지도)
import folium

# 동대문구청 위도 경도 세팅
dongdaemun_map = folium.Map(location = [37.57441938710835, 127.03959808108709], zoom_start = 15) # location[위도, 경도]

dongdaemun_map

In [28]:
# 2) 지도에 마커 표시
# folium.Marker().add_to(지도객체)
# popup 옵션: 마커를 클릭했을 때 표시할 텍스트 설정

import pandas as pd

# 불법 주정차 단속 데이터
dongdaemun_illg_parking_df = pd.read_csv('C:/Users/Administrator/Desktop/dayeong/불법주정차_신고현황_동대문구_filtered.csv')

dongdaemun_illg_parking_df.head()


# zip()
for name, lat, lng in zip(dongdaemun_illg_parking_df['도로명주소'], dongdaemun_illg_parking_df['위도'], dongdaemun_illg_parking_df['경도']):
    #print(f"시설명: {name}, 위도: {lat}, 경도: {lng}")
    folium.Marker([lat, lng], popup = name).add_to(dongdaemun_map)


dongdaemun_map

In [29]:
# 행정지역마다 단속현황 데이터

# 1) 행정동 별 단속현황 건수로 프레임 생성

# '도로명주소' 컬럼에서 동네 데이터 추출
dongdaemun_illg_parking_df['동네'] = dongdaemun_illg_parking_df['도로명주소'].apply(lambda x: x.split()[-3])

# 동네별 데이터 수 구하기
neighborhood_counts = dongdaemun_illg_parking_df['동네'].value_counts()

print("동네별 데이터 수:")
print(neighborhood_counts, type(neighborhood_counts))

동네별 데이터 수:
동네
용신동      48
장안1동     42
장안2동     41
이문2동     35
전농1동     30
회기동      21
이문1동     19
답십리1동    16
제기동      15
청량리동     14
답십리2동     7
휘경1동      5
휘경2동      5
전농2동      1
안암동       1
Name: count, dtype: int64 <class 'pandas.core.series.Series'>


In [30]:
# 시리즈 데이터프레임 변환
illg_parking_df = neighborhood_counts.to_frame()
illg_parking_df

,count
동네,
용신동,48
장안1동,42
장안2동,41
이문2동,35
전농1동,30
회기동,21
이문1동,19
답십리1동,16
제기동,15


In [31]:
# 터미널에서 모듈 설치: pip install pandas geopandas


# 파일 경로
input_file_path = 'C:/Users/Administrator/Desktop/dayeong/map/HangJeongDong_ver20230701.geojson'
output_file_path = 'C:/Users/Administrator/Desktop/dayeong/map/HangJeongDong_Dongdaemun.geojson'

# JSON 파일 읽기
with open(input_file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# '서울특별시 동대문구'가 포함된 데이터 필터링
filtered_features = [feature for feature in data['features'] if '서울특별시 동대문구' in feature['properties']['adm_nm']]

# 필터링된 데이터를 새로운 JSON 구조로 저장
filtered_data = {
    'type': data['type'],
    'features': filtered_features
}

# 필터링된 데이터를 JSON 파일로 저장
with open(output_file_path, 'w', encoding='utf-8') as f:
    json.dump(filtered_data, f, ensure_ascii=False, indent=4)

print(f"필터링된 데이터가 '{output_file_path}' 파일로 저장되었습니다.")


필터링된 데이터가 'C:/Users/Administrator/Desktop/dayeong/map/HangJeongDong_Dongdaemun.geojson' 파일로 저장되었습니다.


In [32]:
# json 파일에 'temp2' 추가하여 저장하기

# 파일 경로
dongdaemun_file_path = 'C:/Users/Administrator/Desktop/dayeong/map/HangJeongDong_Dongdaemun.geojson'
modified_dongdaemun_file_path = 'C:/Users/Administrator/Desktop/dayeong/map/HangJeongDong_Dongdaemun_modified.geojson'

# JSON 파일 읽기
with open(dongdaemun_file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# 'temp2' 키 추가하여 'XX동' 값 저장
for feature in data['features']:
    if 'temp' in feature['properties']:
        temp_value = feature['properties']['temp']
        if '동대문구' in temp_value:
            temp2_value = temp_value.split('동대문구 ')[1]  # '동대문구'를 제외하고 'XX동' 추출
            feature['properties']['temp2'] = temp2_value

# 수정된 데이터를 새로운 JSON 구조로 저장
with open(modified_dongdaemun_file_path, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

print(f"수정된 데이터가 '{modified_dongdaemun_file_path}' 파일로 저장되었습니다.")

수정된 데이터가 'C:/Users/Administrator/Desktop/dayeong/map/HangJeongDong_Dongdaemun_modified.geojson' 파일로 저장되었습니다.


In [33]:
# 행정지역 경계정보 json 데이터 불러오기

dongdaemun_geo_file = 'C:/Users/Administrator/Desktop/dayeong/map/HangJeongDong_Dongdaemun_modified.geojson'
dongdaemun_geo_data = json.load(open(dongdaemun_geo_file, encoding = 'utf-8'))

print(json.dumps(dongdaemun_geo_data, indent = 2, ensure_ascii = False))

{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {
        "adm_nm": "서울특별시 동대문구 회기동",
        "adm_cd": "1106071",
        "adm_cd2": "1123071000",
        "sgg": "11230",
        "sido": "11",
        "sidonm": "서울특별시",
        "temp": "동대문구 회기동",
        "sggnm": "동대문구",
        "adm_cd8": "11060710",
        "temp2": "회기동"
      },
      "geometry": {
        "type": "MultiPolygon",
        "coordinates": [
          [
            [
              [
                127.05202994180435,
                37.59995268207153
              ],
              [
                127.05263341474506,
                37.59942320859219
              ],
              [
                127.05364666696671,
                37.597535331009006
              ],
              [
                127.05460184473503,
                37.59573284360999
              ],
              [
                127.05504867601809,
                37.59357475002139
         

In [55]:
# 새 지도생성 (동대문구 지도)
import folium

# 서울시립대 위치를 기본값으로 세팅
dongdaemun_illg_map = folium.Map(location = [37.58483190086819, 127.057741090489], zoom_start = 14) # location[위도, 경도]

dongdaemun_illg_map

In [65]:
# 2) 행정지역 경계정보를 가지고 있는 데이터 불러오기

dongdaemun_geo_data = json.load(open(dongdaemun_geo_file, encoding = 'utf-8'))


# Choropleth 로 단계구분도 표시
folium.Choropleth(geo_data = dongdaemun_geo_data, # 지도 경계 설정
                 data = illg_parking_df['count'], # 표시하려는 데이터 설정
                 columns = [illg_parking_df.index, illg_parking_df['count']], # 열 지정
                 fill_color = 'YlOrRd', fil_opacity = 0.7, line_opacity = 0.3, # 색깔, 투명도, 선투명도
                 key_on = 'feature.properties.temp2' # jaon에서 행정구 이름 기준으로 설정
                 ).add_to(dongdaemun_illg_map)


dongdaemun_illg_map

In [58]:

# 지도를 HTML 파일로 저장
map_file_path = 'C:/Users/Administrator/Desktop/dayeong/dongdaemun_illg_map.html'
dongdaemun_illg_map.save(map_file_path)

# HTML 파일로 저장된 지도를 브라우저에서 열기
import webbrowser
webbrowser.open(map_file_path)

True

In [64]:
## 군집화
import folium
from folium.plugins import MarkerCluster

# 불법 주정차 단속 데이터
dongdaemun_illg_parking_df = pd.read_csv('C:/Users/Administrator/Desktop/dayeong/불법주정차_신고현황_동대문구_filtered.csv')
'''
map = folium.Map(
    location = [37.58483190086819, 127.057741090489], # 처음 지도가 보여줄 위치를 [위도, 경도] 로 설정
    width = 1500,          # 지도의 너비
    height = 1000,         # 지도의 높이
    zoom_start = 15       # 줌은 최대 18까지 설정 가능
)
'''

df_loc = dongdaemun_illg_parking_df[['위도', '경도', '도로명주소']].dropna()

# 마커 클러스터 객체 생성
cluster = MarkerCluster().add_to(dongdaemun_illg_map) # 지도 정보를 가지고 있는 변수: cluster



for sub_lat, sub_lng, sub_name in zip(df_loc['위도'], df_loc['경도'], df_loc['도로명주소']) :
    # print(sub_lat, sub_lng)
    folium.Marker(
    [sub_lat, sub_lng],
    tooltip=sub_name
    ).add_to(cluster)

map